In [1]:
import numpy as np
import pandas as pd
import random
import math
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.callbacks import Callback, ModelCheckpoint
import sklearn as sk
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
mms = MinMaxScaler()
mas = MaxAbsScaler()
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
PastDays = 21
n_classes = 3
FeatureNums = 29
batch_size = 32
num_epoch = 2

n_filters = 32
kernel_size = [5, 5]
pool_size = [2, 2]

img_cols = PastDays-1
img_rows = FeatureNums
input_shape = (img_cols, img_rows, 1)

In [3]:
def getData(df):
    high = df['high']
    low = df['low']
    change = df['change']
    change_array = []
    delist_array = []
    for i in range(len(df)):
        if high[i] != low[i]:
            delist_array.append(1)
            if change[i] > 10.0:
                change_array.append(10.0)
            elif change[i] < -10.0:
                change_array.append(-10.0)
            else:
                change_array.append(change[i])
        else:
            delist_array.append(0)
    df['change_update'] = pd.DataFrame(change_array)
    df['delist'] = pd.DataFrame(delist_array)
    
    # 去除当日停牌数据
    df = df[df['delist'].isin([1])]
    return df[['open', 'high', 'low', 'close', 'volume', 'amount', 'change_update', 'EMA12', 'EMA26', 
                'MACD', 'MACDsignal', 'MACDhist', 'RSI', 'ROC', 'k', 'd', 'j', 'ROLL_UP',
                'ROLL_MB', 'ROLL_DN', 'diff1', 'diff2', 'OBV', 'm1_yoy', 'm2_yoy', 'dd', 'fix3', 'fix6', 'erd']], df[['date', 'code', 'change_update']]

def DataProcess(df, code):
    data = df
    code = code
    label = df['change_update'].values

    mas.fit(data)
    data = mas.transform(data)                                                                           
    
    label_array = []
    for i in range(len(label)):
        # 平1，跌0、涨2，股票收盘价波动不超过+-0.5%时标签为平
        if label[i] >= 0.5:
            label_array.append(2)
        elif label[i] <= -0.5:
            label_array.append(0)
        else:
            label_array.append(1)
            
    data = pd.DataFrame(data)
    
    return data, label_array, code

def win(data, label, PastDays):
    # 设定滑动窗口大小为1
    newt = []
    newtr = []
    newtl = []
    newl = []
    for i in range(len(data)-PastDays+1):
        for j in range(PastDays):
            if(j < PastDays-1):                
                newt.append(data[i+j])
            else:
                newtl.append(label[i+j-1])
                    
                newtr.append(newt)
                newt = []
    dataset = np.array(newtr)
    labelset = np.array(newtl)
    
    return dataset, labelset

In [4]:
def acc_loss_fig(model_log):
    fig = plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(model_log.history['acc'])
    plt.plot(model_log.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')
    plt.show()

    plt.subplot(2, 1, 2)
    plt.plot(model_log.history['loss'])
    plt.plot(model_log.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.tight_layout()
    plt.show()

In [5]:
stock_code = pd.read_csv("stock_name.csv")
stock_code = stock_code.values
strain_csv = []
stest_csv = []
for i in range(len(stock_code)):
    filepath_train = '2018strain/'
    stock_train = filepath_train+stock_code[i]
    filepath_test = '2018stest/'
    stock_test = filepath_test+stock_code[i]
    strain_csv.append(stock_train)
    stest_csv.append(stock_test)

In [6]:
def data_pre_process(csv):
    dataset = []
    labelset = []
    for stock_item in csv:
        df = pd.read_csv(stock_item[0])
        close = df['close']

        # 拆分训练数据与测试数据并获取标签
        data_num = df.shape[0]
        data, code = getData(df)
        data = data.dropna(axis=0)
        code = code.dropna(axis=0)
        data_1, label_1, code_1 = DataProcess(data, code)
        data_1 = data_1.dropna(axis=0)
        code_1 = code_1.dropna(axis=0)
        data_values = data_1.values

        data_win, label_win = win(data_values, label_1, PastDays)

        num = len(data_win)
        data_random_num = list(range(0, num))
        random.shuffle(data_random_num)

        data_random = []
        label_random = []
        for item in data_random_num:
            data_random.append(data_win[item])
            label_random.append(label_win[item])

        labelset_01 = np_utils.to_categorical(label_random, n_classes)
        
        dataset.extend(data_random)
        labelset.extend(labelset_01)
        
    print("FINISHED")
    return dataset, labelset

In [7]:
def test_pre_process(stock_item):
    df = pd.read_csv(stock_item)

    data_num = df.shape[0]
    data, code = getData(df)
    data = data.dropna(axis=0)
    code = code.dropna(axis=0)
    data_1, label_1, code_1 = DataProcess(data, code)
    data_1 = data_1.dropna(axis=0)
    code_1 = code_1.dropna(axis=0)
    data_values = data_1.values

    data_win, label_win = win(data_values, label_1, PastDays)

    num = len(data_win)
    data_random_num = list(range(0, num))
    random.shuffle(data_random_num)

    data_random = []
    label_random = []
    for item in data_random_num:
        data_random.append(data_win[item])
        label_random.append(label_win[item])

    labelset_01 = np_utils.to_categorical(label_random, n_classes)
    code_1 = pd.DataFrame(code_1)
    print("FINISHED")
    return data_random, labelset_01, code_1

In [ ]:
# 之前生成训练数据和训练标签的代码，现在可以不使用
train_data, train_label = data_pre_process(strain_csv)
train_data = np.array(train_data)
train_data = train_data.reshape(len(train_data), img_cols, img_rows, 1)
train_data = train_data.astype('float32')
train_label = np.array(train_label)

In [10]:
path = []
def model(stock_name, train_data, test_data, train_label, test_label):
    test_data = np.array(test_data)
    X_train = train_data
    X_test  = test_data.reshape(len(test_data), img_cols, img_rows, 1)

    X_test = X_test.astype('float32')
    
    y_train = train_label
    y_test = np.array(test_label)

    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    if X_train.shape[0] == 0:
        return 
    else:
        # 2D model
        model = Sequential()
        model.add(Convolution2D(filters=6, kernel_size=(5,5), padding='valid', input_shape=input_shape, activation='tanh'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Flatten())
        model.add(Dense(120, activation='tanh'))
        model.add(Dense(84, activation='tanh'))
        model.add(Dense(n_classes, activation='softmax'))
        sgd = SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)

        model.summary()

        # 保存模型的权重
        # checkpoint best neural network model only
        filepath = stock_name + ".CNN.weight.best.h5"
        path.append(filepath)
        # checkpoint
        checkpoint = ModelCheckpoint(filepath,
                                                     monitor='val_acc',
                                                     verbose=1,
                                                     save_best_only=True,
                                                     mode='max')

        model.compile(loss=keras.losses.categorical_crossentropy,
                             optimizer=keras.optimizers.Adadelta(),
                             metrics=['accuracy'])

        model.save_weights(filepath)

        model_log = model.fit(X_train, y_train,
                                       batch_size=batch_size,
                                       epochs=num_epoch,
                                       verbose=1,
                                       validation_data=(X_test, y_test),
                                       callbacks=[checkpoint])
        acc_loss_fig(model_log)
    
        model.load_weights(filepath)
        y_pred = np.argmax(model.predict(X_test), axis=1)
        y_pred_01 = np_utils.to_categorical(y_pred, n_classes)
        y_true = y_test
        F1_value = f1_score(y_true, y_pred_01, average='weighted')
        conf_matrix = sk.metrics.confusion_matrix(y_true.argmax(axis=1), y_pred_01.argmax(axis=1))
        score = model.evaluate(X_test, y_test, verbose=0)
        print("F1_score = ", F1_value)
        print("confusion matrix\n", conf_matrix)
        print("Test Score = ", score[0])
        print("Test Accuracy = ", score[1])
        return F1_value, conf_matrix, score[0], score[1], y_pred

In [11]:
def write_csv(data, stock_name):
    file_name = 'trend/'+ stock_name + 'code.csv'
    try:
        data.to_csv(file_name)
    except UnicodeEncodeError:
        print("ERROR")
        
stock_code = []
F1 = []
Test_Score = []
Test_Accuracy = []
for stock_item in stest_csv:
    stock_name = stock_item[0][10: 16]
    print(stock_name)
    test_data, test_label, test_code = test_pre_process(stock_item[0])
    f1_value, conf, ts, ta, y_pred = model(stock_name, train_data, test_data, train_label, test_label)
    # 生产保存每个股票的预测情况trend/code.csv文件，日期、code、预测涨跌、真实涨跌4个字段
    code = pd.DataFrame(columns = ['date', 'code', 'real_change', 'pred_change']) #创建一个空的dataframe
    code['date'] = test_code['date']
    code['code'] = test_code['code']
    code['pred_change'] = pd.DataFrame(y_pred)
    code['real_change'] = pd.DataFrame(test_label)
    write_csv(code, stock_name)
    
    # 存储每支股票的code，F1，accuracy和score
    path.append(filepath)
    stock_code.append(stock_name)
    F1.append(f1_value)
    Test_Score.append(ts)
    Test_Accuracy.append(ta)

000002
FINISHED


AttributeError: 'bytearray' object has no attribute 'shape'

In [ ]:
# 存储所有股票的F1、accuracy、score，存入统计文件score.csv中，有code, F1, score三列
score = pd.DataFrame(columns = ['code', 'F1', 'Test Score', 'Test Accuracy']) 
score['code'] = pd.DataFrame(stock_code)
score['F1'] = pd.DataFrame(F1)
score['Test Score'] = pd.DataFrame(Test_Score)
score['Test Accuracy'] = pd.DataFrame(Test_Accuracy)
score.to_csv('trend/score.csv')

In [ ]:
import pickle
import os.path

# 因为train_data文件过大，超过2G，所以需要用如下方法来存储到pickle中，但通过此方法存储后train data的形式改变了
file_path = "train_data.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
train_data = bytearray(n_bytes)
# write
bytes_out = pickle.dumps(train_data)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])
        
with open(r'train_label.pkl', 'wb') as f:
    pickle.dump(train_label, f, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
import pickle
#打开文件
pickle_open = open('train_data.pkl','rb')
train_data = pickle.load(pickle_open)   #重新导入数据

pickle_open = open('train_label.pkl','rb')
train_label = pickle.load(pickle_open)   #重新导入数据